In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_diagonal_wavelet_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,6494.89,0.01,0.08,0.01,0.01,-0.00,0.00
3,0.01,1622.81,-0.01,-0.01,-0.00,0.00,0.00
4,0.08,-0.01,347.95,-0.00,-0.00,-0.00,-0.00
5,0.01,-0.01,-0.00,53.77,-0.00,0.00,-0.00
6,0.01,-0.00,-0.00,-0.00,7.15,0.00,0.00
7,-0.00,0.00,-0.00,0.00,0.00,0.84,0.00
8,0.00,0.00,-0.00,-0.00,0.00,0.00,0.05


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8
2,1.00000,0.00000,0.00005,0.00001,0.00003,-0.00003,0.00002
3,0.00000,1.00000,-0.00001,-0.00004,-0.00003,0.00000,0.00001
4,0.00005,-0.00001,1.00000,-0.00003,-0.00002,-0.00003,-0.00000
5,0.00001,-0.00004,-0.00003,1.00000,-0.00001,0.00000,-0.00001
6,0.00003,-0.00003,-0.00002,-0.00001,1.00000,0.00002,0.00001
7,-0.00003,0.00000,-0.00003,0.00000,0.00002,1.00000,0.00004
8,0.00002,0.00001,-0.00000,-0.00001,0.00001,0.00004,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.12097144343374626

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.03778953e+06 3.65826251e+05 1.61355134e+04 3.61530706e+02
 5.68071654e+00 5.87549425e-02 9.57096871e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.998980,-0.044256,-0.008816,-0.001358,-0.000179,-0.000022,-0.000001
1,0.043804,0.997992,-0.045256,-0.006623,-0.000879,-0.000103,-0.000006
2,0.010726,0.044523,0.998142,-0.039852,-0.005141,-0.000605,-0.000036
3,0.002062,0.008272,0.039225,0.998157,-0.045207,-0.005188,-0.000312
4,0.000363,0.001461,0.006811,0.044572,0.997199,-0.059558,-0.003506
5,0.000065,0.000258,0.001204,0.007780,0.058943,0.996408,-0.060296
6,0.000008,0.000031,0.000145,0.000937,0.007049,0.059979,0.998174


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist


array([0.00101961, 0.00200778, 0.00185789, 0.00184268, 0.00280133,
       0.00359243, 0.00182569])